In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import psycopg2 as pg
from functions import (
    import_concat, datespace, parse_datetime, parse_stage, reformat_time_string)

pd.set_option('display.max_columns', 500)

In [ ]:
'''
aggregates fitbit data files into sleep, heart rate, activity tables.
export to csv files as tables or,
export to postresSQL database. 
'''

In [2]:
user = 'Beth'
# user = 'SE'
# user = 'SK'
# user = 'BW'
# user = 'SeheeKim'
path = f'/Users/Sehokim/{user}/user-site-export/'
# start_date = '2018-05-04'
start_date = '2017-06-04'
end_date = datetime.strftime(datetime.now(), '%Y-%m-%d')
dates = datespace(start_date, end_date)

print(f'\
      User: {user}\n\
Start Date: {start_date}\n\
  End Date: {end_date}\n\
  Duration: {len(dates)} days')

      User: Beth
Start Date: 2017-06-04
  End Date: 2019-04-20
  Duration: 686 days


In [3]:
# import sleep data
raw_sleep = import_concat(path, 'sleep', dates)

In [5]:
# sleep_df
sleep_df = pd.DataFrame()
sleep_df['Start Time'] = raw_sleep['startTime'].apply(reformat_time_string)
sleep_df['End Time'] = raw_sleep['endTime'].apply(reformat_time_string)
sleep_df['Minutes Asleep'] = raw_sleep['minutesAsleep']
sleep_df['Minutes Awake'] = raw_sleep['minutesAwake']
sleep_df['Number of Awakenings'] = raw_sleep['levels'].apply(parse_stage, args=('wake',))
sleep_df['Time in Bed'] = raw_sleep['timeInBed']
sleep_df['Minutes REM Sleep'] = raw_sleep['levels'].apply(parse_stage, args=('rem',))
sleep_df['Minutes Light Sleep'] = raw_sleep['levels'].apply(parse_stage, args=('light',))
sleep_df['Minutes Deep Sleep'] = raw_sleep['levels'].apply(parse_stage, args=('deep',))
sleep_df.to_csv('/Users/Sehokim/usg/data/sleep_archive.csv', columns=sleep_df.columns, index=None)


In [29]:
# import hr data
raw_hr = import_concat(path, 'heart_rate', dates)

In [38]:
hr_df = pd.DataFrame()
hr_df['date_time'] = raw_hr['dateTime']
hr_df['bpm'] = raw_hr['value'].apply(lambda x: x['bpm'])
hr_df.to_csv('/Users/Sehokim/capstone/data/hr_archive.csv', columns=hr_df.columns, index=None)

In [48]:
# import active minutes
sedentary_minutes = import_concat(path, 'sedentary_minutes', dates)
moderately_active_minutes = import_concat(path, 'moderately_active_minutes', dates)
lightly_active_minutes = import_concat(path, 'lightly_active_minutes', dates)
very_active_minutes = import_concat(path, 'very_active_minutes', dates)

In [53]:
am_df = sedentary_minutes.merge(lightly_active_minutes, on='dateTime', how='inner')\
.merge(moderately_active_minutes, on='dateTime', how='inner')\
.merge(very_active_minutes, on='dateTime', how='inner')
am_df.columns = ['date','sedentary','lightly','moderately','very']
am_df.to_csv('/Users/Sehokim/capstone/data/am_archive.csv', columns=am_df.columns, index=None)

In [266]:
conn = pg.connect(dbname='fitbitdb', host='localhost', port=5432)
engine = create_engine("postgresql://Sehokim@localhost:5432/fitbitdb")
sleep_df.to_sql('sleep', engine, index=True, if_exists='replace', method='multi')
hr_df.to_sql('heart_rate', engine, if_exists='replace', method='multi')
am_df.to_sql('active_minute', engine, if_exists='append', method='multi')


KeyboardInterrupt: 